# Hello, Welcome!

This Notebook uses Machine Learning / PyTorch for recognizing handwritten numbers, but can easily be modfied for other class recognition purposes. You may use this as a kickstarter or skeleton for your own purposes, or steal this entirely. Just call me out when you do so please.  

This notebook is based on the Pytorch tutorial by Patrick Loeber:  
https://www.youtube.com/watch?v=c36lUUr864M

I'm using the Dataset from this link:  
https://www.kaggle.com/datasets/jcprogjava/handwritten-digits-dataset-not-in-mnist

### Known Issues:
- Output flickers for Visual Studio. However, this seems to be an issue with VS and might be impossible to fix from my side.

### If you are using Windows:  
Copy/Paste the line below into the Windows cmd console if tensorboard won't start or is giving you a hard time in general.
Kills the tensorboard process, which can be really tough and persists to run even after restart of your machine.

    del /q %TMP%\.tensorboard-info\*

Also, Microsoft Defender is slowing down Dataloading significantly.

### If you are using Tensorboard:  
Tensorboard creates new folders in your working directory. Make sure you have writing permissions or turn tensorboard off by setting

    tb_analytics = False. 

Don't forget to delete everything in the folder once you're done here!

### If you are using Jupyter Notebooks:  
Use this link to start a tensorboard session in your browser:  
http://localhost:6006/


In [ ]:
### ALL THE IMPORTS ###
#######################

# PyTorch imports
import torch as tc
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, Subset
from torch.optim import lr_scheduler
from torch.multiprocessing import Pool

import torchvision as tv
import torchvision.io as io

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# PIL Imports
from PIL import Image
from PIL import ImageStat
from PIL import ImageOps
from PIL import ImageShow
from PIL import ImageFilter

# Standard pkg imports
import sys
import os
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output
from threading import Thread as Thread
from threading import Event as Event
import re

# Tensorboard imports
import tensorboard
from tensorboard import notebook
from torch.utils.tensorboard import SummaryWriter
logdir =  './runs/'                 # dir in which to save run data
writer = SummaryWriter(logdir)      # init tensorboard data writer

%load_ext tensorboard
%tensorboard --logdir=runs
dir_counter = 0                     # counter for setting up tensorboard folders; different training runs will be saved in different folders

clear_output()

# CUDA
if tc.cuda.is_available():
    device = tc.device("cuda")
else:
    device = tc.device("cpu")

print(f"CUDA is available: {tc.cuda.is_available()}")

import RecognitionNN_utils as utils

In [ ]:
#### PREPARE DATA ####
###### - 0.1 - #######
if __name__ == "__main__":

        # Adapter from your Dataset to Dataloader and thus specific for each Dataset.
        # Microsoft Defender might slow things down here significantly. Take a look at your task manager.

        datapath = "D:\cempos python scripts\machine learning\Doodles\quickdraw_dataset_png"
        train_test_ratio = 0.01                                     # define ratio of train/total samples


        ID_list = []
        counter = 0
        for root, dirs, files in os.walk(datapath):
                for idx, file in enumerate(files):
                        ID_list.append(idx + counter)
                counter += len(files)
        del counter

        label_list = []
        with open('label_list.txt') as f:
                for line in f:
                        label_list.append(line.rstrip())

        quickdraw_dataset = utils.Quickdraw_Dataset(ID_list, label_list)
        quickdraw_trn, quickdraw_tst = quickdraw_dataset.split_trn_tst(train_test_ratio)

        del ID_list
        del label_list
        del quickdraw_dataset

In [ ]:
##### DATALOADER #####
###### - 0.2 - #######
if __name__ == "__main__":

    # Shove both training and test lists into the dataloader.

    batch_size = 8                                                  # define batch_size

    batches_trn = DataLoader(dataset = quickdraw_trn,                    # samples input
                        batch_size = batch_size,
                        shuffle = False,                           # no need to shuffle, we already did that
                        num_workers = 4,                           # strongly recommended to keep = 0
                        persistent_workers = True,
                        pin_memory = True)                         # prevent loading into memory after every epoch

    batches_tst = DataLoader(dataset = quickdraw_tst,
                        batch_size = batch_size,
                        shuffle = False,
                        num_workers = 4,
                        persistent_workers = True,
                        pin_memory = True)

    print('Done.')

In [ ]:
##### DEFINE MODEL ####
######## - 1 - ########
if __name__ == "__main__":

    model = utils.ConvNN(quickdraw_trn.__getitem__(0)[0].shape,
                fc1_dim = 64,
                fc2_dim = 64,
                output_dim = len(quickdraw_trn.label_list)
                )

    # Optional Block for loading in a model and/or model-state from disk
    # filepath = './'
    # filename = "Tutorial_CNN.pth"
    # model = tc.load(f"{filepath}{filename}").to(device)
    # model.load_state_dict(tc.load(f"{filepath}{filename}_state_dict"))

    ###### LOSS/COST ######
    ###### OPTIMIZER ######
    ###### SCHEDULER ######
    ######## - 2 - ########

    criterion = nn.CrossEntropyLoss()

    optimizer = tc.optim.Adam(model.parameters(),
                            lr = .001)                                     # define initial learning rate

    step_lr_scheduler = lr_scheduler.StepLR(optimizer,
                                            step_size = 1,                  # diminish learning rate every n-th epoch...
                                            gamma = .1)                     # ...by this diminishing factor

    print('Done.')

In [ ]:
###### TRAINING #######
######## - 3 - ########
if __name__ == "__main__":

    # Optional Block for using tensorboard
    tb_analytics = False


    if tb_analytics:
        logdir = f"./runs/{dir_counter}/"
        writer = SummaryWriter(logdir)
        utils.tb_write_model(model, batches_trn)
        dir_counter += 1


    # Training Loop. 
    # Note that loading the batches into memory might take a few minutes. Take a look at your task manager.
    model_trn = utils.training_loop(n_epochs = 1,                             # See func definition for input details
                            print_fps = 5.,
                            model = model,
                            batches_trn = batches_trn,
                            criterion = criterion,
                            optimizer = optimizer,
                            scheduler = step_lr_scheduler,
                            tb_analytics = tb_analytics)

In [ ]:
####### TESTING #######
######## - 4 - ########
if __name__ == "__main__":

    # Testing Loop.
    # Again, loading batches into memory may take a few minutes. Stay strong.

    # device = tc.device("cpu")                                           # optional, but runs faster on cpu for some reason
    pool = Pool()
    accuracy = utils.validation_loop(print_fps = 5.,                          # See func definition for input details
                            print_miss = False,
                            model = model_trn.to(device),
                            batches_tst = batches_tst)

    # if tc.cuda.is_available():                                          # change back to gpu
    #     device = tc.device("cuda")
    # else:
    #     device = tc.device("cpu")
    # model_trn = model_trn.to(device)

In [ ]:
##### SAVE MODEL ######
######## - 5 - ########
if __name__ == "__main__":

    # Set filepath and model name
    filepath = './'
    model_name = 'Tutorial_CNN'

    tc.save(model_trn, f"{filepath}{model_name}.pth")                               # Save the whole model and/or...
    tc.save(model_trn.state_dict(), f"{filepath}{model_name}_state_dict.pth")       # ...save only the the model state

    print('Done.')

In [ ]:
# I'm just here so that the Screen doesn't flick when printing images

















